# Comparison using kerchunk / datatree

In [ ]:
from datatree import DataTree
from datatree import map_over_subtree
import dask.bag as db
import xarray as xr
import pandas as pd
import dask
import xclim
import thermofeel as tf
import numpy as np
from distributed import Client
from fsspec.implementations.reference import ReferenceFileSystem
import kerchunk
import coiled
from utils import wbgt, load_elev, adjust_pressure

In [ ]:
import coiled

cluster = coiled.Cluster(n_workers=10)
client = cluster.get_client()

In [ ]:
# Read the reference catalog into a Pandas DataFrame
cat_df = pd.read_csv(
    "s3://carbonplan-share/nasa-nex-reference/reference_catalog_nested.csv"
)
# Select only ssp245 && HISTORICAL!
ssp245_historical_catalog = cat_df[cat_df["ID"].str.contains("ssp245|historical")]

ssp245_historical_catalog = ssp245_historical_catalog.iloc[0:20]
# Convert the DataFrame into a dictionary
catalog = ssp245_historical_catalog.set_index("ID").T.to_dict("records")[0]

# invalid? float is not iterable - this GCM ref is invalid
# del catalog["GISS-E2-1-G/ssp245"]

In [ ]:
def load_ref_ds(gcm_scenario: str, url: str):
    storage_options = {
        "remote_protocol": "s3",
        "target_protocol": "s3",
        "lazy": True,
        "skip_instance_cache": True,
    }  # options passed to fsspec
    open_dataset_options = {"chunks": {}}  # opens passed to xarray
    ds = xr.open_dataset(
        url,
        engine="kerchunk",
        storage_options=storage_options,
        open_dataset_options=open_dataset_options,
    )

    if set(
        [
            "huss",
            "tasmax",
            "tas",
        ]
    ).issubset(set(list(ds))):
        ds = ds[["huss", "tasmax", "tas"]]
        # adding the gcm/scenario combo to attrs for later down the pipeline
        ds.attrs["gcm_scenario"] = gcm_scenario
        return {gcm_scenario: ds}


# convert catalog dict to tuples for dask bag starmap
catalog_tuple_list = list(catalog.items())
# Create a bag from tuple catalog
cat_bag = db.from_sequence(catalog_tuple_list)
# apply load_ref_ds to catalog tuple bag
task_bag = filter(None, cat_bag.starmap(load_ref_ds).compute())

# convert list of dicts to dict
catalog_computed = {k: v for element in task_bag for k, v in element.items()}

In [ ]:
dt = DataTree.from_dict(catalog_computed)
elev = load_elev()

In [ ]:
def calc_wbgt(ds):
    ds = ds.to_dataset()
    ds = ds.isel(time=slice(0, 365))
    # calculate elevation-adjusted pressure
    ds["ps"] = xr.apply_ufunc(adjust_pressure, ds["tas"], elev, dask="allowed").rename(
        {"elevation": "ps"}
    )["ps"]
    ds["ps"].attrs["units"] = "Pa"
    ds["hurs"] = xclim.indices.relative_humidity(
        tas=ds["tasmax"], huss=ds["huss"], ps=ds["ps"]
    )
    ds["tasmax"].attrs = {}

    # windspeed assumption of 0.5 m/s (approximating shaded/indoor
    # conditions)
    ds["sfcWind"] = (ds["tas"] - ds["tas"]) + 0.5
    ds["WBT"] = tf.thermofeel.calculate_wbt(ds["tasmax"] - 273.15, ds["hurs"])

    ds["BGT"] = tf.thermofeel.calculate_bgt(ds["tasmax"], ds["tasmax"], ds["sfcWind"])
    ds["WBGT"] = wbgt(ds["WBT"], ds["BGT"], ds["tasmax"] - 273.15)
    ds["WBGT"].attrs["units"] = "degC"
    ds = ds[["WBGT"]]
    output = (
        f"s3://carbonplan-scratch/TEMP_NASA_NEX/wbgt-shade-"
        f"gridded/years/{ds.attrs['gcm_scenario']}.zarr"
    )

    return ds.to_zarr(output, consolidated=True, compute=True, mode="w")


ds_list = [ds for ds in dt.leaves if ds.dims]

bag = db.from_sequence(ds_list, npartitions=len(ds_list)).map(calc_wbgt)

In [ ]:
bag.compute()